In [1]:
import importlib, sys
# script 경로 window용
sys.path.append(r"C:\Users\bong2\OneDrive\Python_Codes\00_python_scripts") 
from glob import glob
import pandas as pd
import conn_db
import helper, trade_cleaner

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)  

In [24]:
importlib.reload(trade_cleaner)

<module 'trade_cleaner' from 'C:\\Users\\bong2\\OneDrive\\Python_Codes\\00_python_scripts\\trade_cleaner.py'>

In [25]:
all_files = glob(conn_db.get_path('국가별_수출입_raw') + "*.xls")
df = pd.concat([pd.read_excel(file, skiprows=4)
                for file in all_files], axis=0).drop_duplicates()

C:\Users\bong2\Anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [26]:
df.head(3)

,기간,국가명,수출건수,수출금액,수입건수,수입금액,무역수지
0,총계,NaN,"45,412,781","2,699,752,863","46,980,544","2,542,820,223","156,932,639"
1,2000.01,미국,"45,538","2,609,788","52,999","2,217,025","392,763"
2,2000.01,일본,"35,855","1,551,722","63,151","2,357,231","-805,509"


In [27]:
filt = df['기간'] != '총계'
df = df.loc[filt]
names = {'기간': '날짜',
        '국가명': '국가',
        '수출건수': '수출건수 (건)',
        '수입건수': '수입건수 (건)',
        '수출금액': '수출금액 (천$)',
        '수입금액': '수입금액 (천$)',
        '무역수지': '무역수지 (천$)'}
df.rename(columns=names, inplace=True)
df['날짜'] = pd.to_datetime(df['날짜'])

In [28]:
df.head(3)

,날짜,국가,수출건수 (건),수출금액 (천$),수입건수 (건),수입금액 (천$),무역수지 (천$)
1,2000-01-01,미국,"45,538","2,609,788","52,999","2,217,025","392,763"
2,2000-01-01,일본,"35,855","1,551,722","63,151","2,357,231","-805,509"
3,2000-01-01,중국,"27,336","1,235,215","31,458","1,008,166","227,050"


In [29]:
matcher = ['건수', '금액', '무역']
all_cols = df.columns.tolist()
value_cols = [col for col in all_cols if any(
    prcnt in col for prcnt in matcher)]

for col in value_cols:
    df[col] = pd.to_numeric(df[col].str.replace(',', ''))

In [30]:
df.head(3)

,날짜,국가,수출건수 (건),수출금액 (천$),수입건수 (건),수입금액 (천$),무역수지 (천$)
1,2000-01-01,미국,45538,2609788,52999,2217025,392763
2,2000-01-01,일본,35855,1551722,63151,2357231,-805509
3,2000-01-01,중국,27336,1235215,31458,1008166,227050


In [31]:
df = trade_cleaner.add_units(df)
df.head(3)

,날짜,국가,수출건수 (건),수출금액 (천$),수입건수 (건),수입금액 (천$),무역수지 (천$),수출금액 (백만$),수입금액 (백만$),무역규모 (백만$),무역수지 (백만$),수출금액 (억$),수입금액 (억$),무역규모 (억$),무역수지 (억$),무역규모 (천$),수출 건당 (천$/건),수입 건당 (천$/건)
1,2000-01-01,미국,45538,2609788,52999,2217025,392763,2609.788,2217.025,4826.813,392.763,26.09788,22.17025,48.26813,3.92763,4826813,57.310115,41.831450
2,2000-01-01,일본,35855,1551722,63151,2357231,-805509,1551.722,2357.231,3908.953,-805.509,15.51722,23.57231,39.08953,-8.05509,3908953,43.277702,37.326899
3,2000-01-01,중국,27336,1235215,31458,1008166,227049,1235.215,1008.166,2243.381,227.049,12.35215,10.08166,22.43381,2.27049,2243381,45.186384,32.048001


In [37]:
cols_long = ['날짜', '국가', '수출건수 (건)', '수출금액 (천$)', '수입건수 (건)', '수입금액 (천$)', '무역수지 (천$)',
             '무역규모 (천$)', '수출 건당 (천$/건)', '수입 건당 (천$/건)',
             '수출금액 (백만$)', '수입금액 (백만$)', '무역규모 (백만$)', '무역수지 (백만$)',
             '수출금액 (억$)', '수입금액 (억$)', '무역규모 (억$)', '무역수지 (억$)', ]

cols_short = ['날짜', '국가', '수출건수 (건)', '수출금액 (천$)', '수입건수 (건)',
              '수입금액 (천$)', '무역수지 (천$)','무역규모 (천$)', 
              '수출 건당 (천$/건)', '수입 건당 (천$/건)']

In [38]:
# 저장
conn_db.export_(df[cols_long], '수출입_국가별')
for sheet in ['DB_수출입_국가별','DB_수출입_국가별_대시보드']:
    conn_db.to_(df[cols_short], sheet, 'import')

C:\Users\bong2\OneDrive\Python_Codes\00_python_scripts\conn_db.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(axis=1, how='all', inplace=True)
C:\Users\bong2\Anaconda3\lib\site-packages\pygsheets\worksheet.py:1366: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype('unicode').replace('<NA>', nan)
